In [1]:
# Setup:
# 1. Uninstall previous version if already installed
%pip uninstall pyafmreader -y
# 2. Install pyafmreader from github 
# (temporary, when 1.0 releases it will be available via pip)
%pip install git+https://github.com/jlopezalo/pyafmreader.git

Found existing installation: pyafmreader x.x.x
Uninstalling pyafmreader-x.x.x:
  Successfully uninstalled pyafmreader-x.x.x
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/jlopezalo/pyafmreader.git to /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-req-build-_cazlmms
  Running command git clone --filter=blob:none --quiet https://github.com/jlopezalo/pyafmreader.git /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-req-build-_cazlmms
  Resolved https://github.com/jlopezalo/pyafmreader.git to commit f750f951e150848e7a69cc8b4182f89181c9a822
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/TkTech/fasterzip.git to /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-install-_6bnlcog/fasterzip_929f50db6aff42b9b463455330e33394
  Running command git clone --filter=blob:none --quiet https://github.com/TkTech/fasterzip.git /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-install-_6bnlc

In [2]:
# 1. Import pyafmreader loadfile and matplotlib
from pyafmreader import loadfile
import matplotlib.pyplot as plt
import os

# Get documentation about loadfile function
help(loadfile)

Help on function loadfile in module pyafmreader.pyafmreader:

loadfile(filepath)
    Load AFM file. 
    
    Supported formats:
        - JPK --> .jpk-force, .jpk-force-map, .jpk-qi-data
        - JPK Thermal --> .tnd
        - NANOSCOPE --> .spm, .pfc, .00X
        - UFF --> .uff
    
            Parameters:
                    filepath (str): Path to the file.
            
            Returns:
                    If JPK, NANOSCOPE OR UFF:
                        UFF (uff.UFF): Universal File Format object containing loaded data.
                    If JPK Thermal:
                        Amplitude (m^2/V) (np.array),
                        Frequencies (Hz) (np.array),
                        Fit-Data (m^2/V) (np.array),
                        Parameters (dict)



In [3]:
# 2. Load one of the test files
# NANOSC_PFC_PATH = '../tests/testfiles/08171528.0_00001.pfc'

NANOSC_PFC_PATH = '../tests/testfiles/calibration3.000'

NANOSC_PFC_FILE = loadfile(NANOSC_PFC_PATH)

type(NANOSC_PFC_FILE)

pyafmreader.uff.UFF

In [4]:
# 3. Get file metadata
metadata = NANOSC_PFC_FILE.filemetadata

for key, item in metadata.items():
    print(f"{key} : {item}\n") 

file_path : ../tests/testfiles/calibration3.000

Entry_filename : calibration3.000

file_size_bytes : 45056

file_type : .000

UFF_code : _1_2_3_4_5

Entry_UFF_version : 0

version : 0x09010201

instru : BioScope Catalyst

scanner : 1B923.scn

zscan_sens_nmbyV : 106.3

force_volume : 0

xoffset_nm : 0.0

yoffset_nm : 0.0

piezo_nb_sampsline : 256

xy_closed_loop : Off

z_closed_loop : Off

PFC_nb_samppoints : 1024

NEW_sync_dist : 371

QNM_sync_dist : 371

PFC_amp : 500.0

ramp_size_V : 9.407338

PFC_freq : 0.5

peakforce : 0

defl_sens_nmbyV : 30.3677

sens_z_sensor : 1280.381

scan_rate_Hz : 1.02796

speed_forward_Vbys : 19.3407

speed_reverse_Vbys : 19.3407

nb_point_approach : 508

nb_point_retract : 512

trigger_mode : Relative

FDC_nb_sampsline : 16

defl_sens_corr : 1.08

defl_sens_Vbybyte : 0.000375

data_offset : 40960

FDC_data_length : 2048

byte_per_pixel : 2

spring_const_Nbym : 0.0677954

z_scale_Vbybyte : 24.576

Entry_tot_nb_curve : 1

ramp_size_nm : 1000.0000293999999


In [5]:
# 4. Get force volume piezo image and plot it
piezoimg = NANOSC_PFC_FILE.getpiezoimg()

plt.figure(figsize=(10,10))
plt.imshow(piezoimg[:, :, 0], cmap='afmhot', origin='lower')
plt.show()

TypeError: 'NoneType' object is not subscriptable

<Figure size 1000x1000 with 0 Axes>

In [6]:
# 5. Get force curve at index 0
FC = NANOSC_PFC_FILE.getcurve(0)

type(FC)

UnboundLocalError: local variable 'FC' referenced before assignment

In [ ]:
help(FC.preprocess_force_curve)

In [ ]:
# 6. Preprocess curve with the deflection sens in the header
defl_sens = metadata['defl_sens_nmbyV'] / 1e09 # nm/V --> m/V
FC.preprocess_force_curve(defl_sens, metadata['height_channel_key'])
plt.figure(figsize=(10,5))
for segid, segment in FC.get_segments():
    plt.plot(segment.zheight, segment.vdeflection, label=f'Segment: {segid}')
plt.xlabel('zheight [m]')
plt.ylabel('vdeflection [m]')
plt.legend()
plt.show()

In [ ]:
# 7. Get Force vs Indentation curve
poc = [0.5 * 1e-8, 0.0] # in nm
spring_k = metadata['spring_const_Nbym']
FC.get_force_vs_indentation(poc, spring_k)
plt.figure(figsize=(10,5))
for segid, segment in FC.get_segments():
    plt.plot(segment.indentation, segment.force, label=f'Segment: {segid}')
plt.axhline(y=0, color='k', linestyle='--')
plt.axvline(x=0, color='k', linestyle='--')
plt.xlabel('Indentation [m]')
plt.ylabel('Force [N]')
plt.legend()
plt.show()

In [ ]:
# 8. Get segment 0
FC_segments = FC.get_segments()
_, segment_0 = FC_segments[0]
type(segment_0)

In [ ]:
# 9. Plot only segment 0
plt.figure(figsize=(10,5))
plt.plot(segment_0.zheight, segment_0.vdeflection, label=f'Segment: 0')
plt.xlabel('zheight [nm]')
plt.ylabel('vdeflection [nm]')
plt.legend()
plt.show()

In [ ]:
# 9. Get force vs indentation for segment 0
plt.figure(figsize=(10,5))
plt.plot(segment_0.indentation, segment_0.force, label=f'Segment: 0')
plt.xlabel('zheight [nm]')
plt.ylabel('vdeflection [nm]')
plt.legend()
plt.show()